In [1]:
import sys; sys.path.append("../")
import pandas as pd

from clean.st_tammany_so_pprr import clean
from clean.st_tammany_so_cprr import clean as clean_cprr, clean_depts

In [2]:
pprr = clean()
pprr

../lib/clean.py:152: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\


,employee_id,last_name,first_name,birth_year,rank_code,rank_desc,pay_group,hire_year,hire_month,hire_day,term_year,term_month,term_day,uid
0,1,divincenti,frank,1923,CG01,school crossing guard,PSAF,1984,09,10,,,,6a22a802ff4f5072bdb2ef289c443e26
1,2,husser,linda,1949,EX04,consultant,AUX,2010,08,03,2016,06,30,af8f11a2cbd55d7e4de29ffaafffbe0d
2,3,jenkins,dickie,1955,MJ3,mj3 corr/comm/crt sec,AUX,1980,11,14,2006,02,03,70e5ff989ccf16676d48d668a08fcbd1
3,4,ranatza,gary,1964,M101,lieutenant protective services,PSAF,1996,09,26,2019,12,12,16e8d29524c6e31728c219dba5945aeb
4,5,kelly,robert,1926,CG01,school crossing guard,PSAF,1994,08,16,2001,10,31,3b029846b7b4037fb1ccf5a41ace73d0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3117,80996,beyer,justin,1974,RESV,reserve deputies,NaN,2020,01,14,,,,85f7e3e5d76557cc71dfbb8b84b56731
3118,80997,barberito,christopher,1990,RESV,reserve deputies,NaN,2020,01,28,,,,892431106b06f986346afabf739f6169
3119,80998,scarlato,michael,1980,RESV,reserve deputies,NaN,2020,02,13,,,,e937a1d07bbe01d1c467a04e6d985cf6
3120,80999,rogers,keith,1962,RESV,reserve deputies,NaN,2020,06,16,,,,84821a28cb1e0cec69382c7ec578f973


In [3]:
pprr[(pprr.first_name == 'charles') & (pprr.last_name.str.startswith('smith'))]

,employee_id,last_name,first_name,birth_year,rank_code,rank_desc,pay_group,hire_year,hire_month,hire_day,term_year,term_month,term_day,uid
116,119,smith jr,charles,1948,M202,corrections capt,AUX,1981,10,28,2002,04,11,854e9bbf9f908b776b5004c50eb61712
795,922,smith,charles,1963,A101,deputy/admin auxillary,AUX,2017,09,26,2019,12,18,9a8ad62809baf86d2ed47d49543a99d5
1376,1666,smith,charles,1962,P101,deputy,PSAF,2006,08,02,,,,ea1566bfddd6ffdb639ae513803e5c04


In [4]:
cprr = clean_cprr()
cprr

../clean/st_tammany_so_cprr.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col] = df[col].str.replace(r'(\d)\r\n(\d)', r'\1\2')\


,full_name,incident_date,department_code,infraction,first_name,middle_name,last_name
0,Cody Talley,1/16/2015,36,DR.07:0200.400 - Maintenance and care of Vehi...,cody,,talley
1,Shawn Barrow,1/18/2015,20,DR.03:09.970.01 (a) - Job Performance - wanton...,shawn,,barrow
2,Kendall Twillie,2/7/2015,20,DR.03:09.934.01 - Failure to report - refusal ...,kendall,,twillie
3,Robert Jones,2/13/2015,12,DR.03:09.500.01 - disruptive or abusive behavi...,robert,,jones
4,Ryan Suhre,2/27/2015,36,SOP.1100:02.000- Property and Evidence Control...,ryan,,suhre
...,...,...,...,...,...,...,...
273,"Charles ""Chuck"" Smith","11/5,6,8,13,14,15,26/2019",20,DR.03:09.932.01 Failure to follow supervisor i...,charles,,smith
274,Jonathan Thornhill,12/19/2019,34,DR.03:09.957.01 Willful disregard of Agency ru...,jonathan,,thornhill
275,Timothy Scanlan,1/1/2020,34,DR. 03:09.977.01 Sleeping on duty,timothy,,scanlan
276,Alexandria Payne,Oct-Dec 2019,20,DR.03:09.916.01C Unsatifactory work performance,alexandria,,payne


In [5]:
depts = clean_depts()
depts[depts.department_code.duplicated(keep=False)]

,department_code,department_desc
15,20,Jail Fund
16,20,Commissary Fund
18,20,St. Tammany Parish Government
19,20,Jail K9


In [7]:
cprr.department_code.unique()

array(['36', '20', '12', '10', '18', '15', '17', '28', '07', '26', '34',
       '35', '39', '47', '19', '42', '09', '24', '', '37', '22', '29',
       '06', '30', '45', '25', '50', '40', '21', '71'], dtype=object)